In [1]:
import streamlit as st
import duckdb
import pandas as pd


In [ ]:
#Connect to DuckDB and retrieve data:
@st.cache
def load_data():
    con = duckdb.connect(database=':memory:')
    con.execute("INSTALL sqlite; LOAD sqlite;")
    con.execute("PRAGMA sqlite._sqlite3.dir='./';")  # Modify the path as needed
    con.execute("CALL sqlite_attach('food_recipes.db', 'main');")
    
    recipes_df = con.execute("SELECT * FROM recipes").df()
    ingredients_df = con.execute("SELECT * FROM ingredients").df()
    return recipes_df, ingredients_df

recipes_df, ingredients_df = load_data()


In [ ]:
#Build the Streamlit UI:
st.title("Recipe Finder")
st.write("Enter the ingredients you have at home, and we'll show you possible recipes.")

user_ingredients = st.text_input("Enter ingredients separated by commas (e.g., tomato, cheese, pasta):")


In [ ]:
#Filter and display recipes:
def find_recipes(user_ingredients, recipes_df, ingredients_df):
    user_ingredients_set = set([ingredient.strip().lower() for ingredient in user_ingredients.split(',')])
    possible_recipes = []

    for index, row in recipes_df.iterrows():
        recipe_id = row['id']
        recipe_ingredients = ingredients_df[ingredients_df['recipe_id'] == recipe_id]['ingredient'].tolist()
        recipe_ingredients_set = set([ingredient.lower() for ingredient in recipe_ingredients])
        
        if user_ingredients_set.issubset(recipe_ingredients_set):
            possible_recipes.append(row['title'])
    
    return possible_recipes

if st.button("Find Recipes"):
    if user_ingredients:
        possible_recipes = find_recipes(user_ingredients, recipes_df, ingredients_df)
        if possible_recipes:
            st.write("Here are some recipes you can make:")
            for recipe in possible_recipes:
                st.write(f"- {recipe}")
        else:
            st.write("No recipes found with the given ingredients.")
    else:
        st.write("Please enter some ingredients.")


In [ ]:
streamlit run app.py
